In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from time import sleep
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import pandas as pd
import pickle
import psutil
import numpy as np

from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton

In [2]:
opts = FirefoxOptions()
opts.add_argument("--width=400")
opts.add_argument("--height=800")
opts.add_argument("--disable-popup-blocking")

service = Service('/snap/bin/firefox.geckodriver')
opts.set_preference("profile.managed_default_content_settings.images", 2)
opts.set_preference("permissions.default.image", 2)

#opts.add_argument('--headless')

In [3]:
try:
  nv.quit()
  del nv
except:
    pass
nv = webdriver.Firefox(options = opts, service=service)
# install ublock
nv.install_addon('temp/ublock.xpi', temporary=True)

nv.get('https://finance.yahoo.com/quote/%5EGSPC/news/')
sleep(4)

In [4]:
main = nv.find_element(By.XPATH, '//main[starts-with(@id, "nimbus-app")]')

mainContent = nv.find_element(By.XPATH, '//section[starts-with(@class, "mainContent")]//section[starts-with(@class, "container")]')
container = mainContent.find_element(By.XPATH, '//div[starts-with(@class, "filtered-stories")]')

article_list = container.find_elements(By.XPATH, './/li[starts-with(@class, "stream-item")]')
article_list = [article for article in article_list if 'native-ad' not in article.get_attribute('class')]
html = article_list[6].get_attribute('outerHTML')
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

title = soup.find('h3').text
subtitle = soup.find('p').text
# find date by div starts with class 'publishing'
publishing = soup.find('div', class_='publishing').text
publisher = publishing.split('•')[0].strip()
date = publishing.split('•')[1].strip()
taxonomies = soup.find('div', class_='taxonomy-links').text


print(title)
print(subtitle)
print(publisher)
print(date)
#print(taxonomies.strip().split('  '))

taxonomies_list = []
# ['^GSPC', '-0.19%', 'BRK-B', '-0.96%']
# for every pair, we have a stock and its change
print(taxonomies.split('  '))
for i in range(0, len(taxonomies.strip().split('  ')), 2):
    if i+1 < len(taxonomies.strip().split('  ')):
        taxonomies_list.append([taxonomies.strip().split('  ')[i], taxonomies.strip().split('  ')[i+1]])
    else:
        taxonomies_list.append([taxonomies.strip().split('  ')[i], ''])
        

print(taxonomies_list)

Dow Jones Futures: Stock Market Bulls In Control; Meta, Apple, Tesla In Buy Areas
The stock market is sending a clear bullish signal, with titans Meta, Apple and Tesla among dozens of buys.
Investor's Business Daily
8 hours ago
[' ^DJI', '+0.09%', 'TSLA', '', 'META', '', '']
[['^DJI', '+0.09%'], ['TSLA', ''], ['META', '']]


In [12]:
main = nv.find_element(By.XPATH, '//main[starts-with(@id, "nimbus-app")]')

mainContent = nv.find_element(By.XPATH, '//section[starts-with(@class, "mainContent")]//section[starts-with(@class, "container")]')
container = mainContent.find_element(By.XPATH, '//div[starts-with(@class, "filtered-stories")]')

#article_list = container.find_elements(By.XPATH, './/li[starts-with(@class, "stream-item")]')
#article_list = article_list[:5]
#article_list = [article for article in article_list if 'native-ad' not in article.get_attribute('class')]

unique_articles = []
article_titles = []
article_list = container.find_elements(By.XPATH, './/li[starts-with(@class, "stream-item")]')

article_child = []
for i in range(1000):
    try:
        print(f'--- iteração {i} ---')

        article_list = container.find_elements(By.XPATH, './/li[starts-with(@class, "stream-item")]')
        article_list = [article for article in article_list if 'native-ad' not in article.get_attribute('class')]
        for article in article_list:
            html = article.get_attribute('outerHTML')
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find('h3').text
            if title not in article_titles:
                #print(title)
                subtitle = soup.find('p').text
                publishing = soup.find('div', class_='publishing').text
                publisher = publishing.split('•')[0].strip()
                date = publishing.split('•')[1].strip()
                taxonomies = soup.find('div', class_='taxonomy-links').text
                taxonomies_list = []
                for i in range(0, len(taxonomies.strip().split('  ')), 2):
                    if i+1 < len(taxonomies.strip().split('  ')):
                        taxonomies_list.append([taxonomies.strip().split('  ')[i], taxonomies.strip().split('  ')[i+1]])
                    else:
                        taxonomies_list.append([taxonomies.strip().split('  ')[i], ''])
                article_titles.append(title)
                article_child = [title, subtitle, publisher, date, taxonomies_list]
                unique_articles.append(article_child)
        main.send_keys(Keys.PAGE_DOWN)

        article_child = []
        print(f'article count: {len(unique_articles)}')
        sleep(2)
    except Exception as e:
        print("error: ", e)
        print()
        continue
    '''


        title = article.find_element(By.XPATH, './/h3').text
        print(title)
        print('---')
        main.send_keys(Keys.PAGE_DOWN)
        article_list = [article for article in article_list if 'native-ad' not in article.get_attribute('class')]
        index_final = len(article_list)
        sleep(2)
        del article, title
    except Exception as e:
        print("error: ", e)
        continue
    '''

--- iteração 0 ---
article count: 40
--- iteração 1 ---
article count: 40
--- iteração 2 ---
article count: 40
--- iteração 3 ---
article count: 40
--- iteração 4 ---
article count: 40
--- iteração 5 ---
article count: 40
--- iteração 6 ---
article count: 40
--- iteração 7 ---
article count: 40
--- iteração 8 ---
article count: 40
--- iteração 9 ---
article count: 40
--- iteração 10 ---
article count: 40
--- iteração 11 ---
article count: 40
--- iteração 12 ---
article count: 40
--- iteração 13 ---
article count: 40
--- iteração 14 ---
article count: 40
--- iteração 15 ---
article count: 60
--- iteração 16 ---
article count: 60
--- iteração 17 ---
article count: 60
--- iteração 18 ---
article count: 60
--- iteração 19 ---
article count: 60
--- iteração 20 ---
article count: 60
--- iteração 21 ---
article count: 60
--- iteração 22 ---
article count: 60
--- iteração 23 ---
article count: 80
--- iteração 24 ---
article count: 80
--- iteração 25 ---
article count: 80
--- iteração 26 ---
ar

KeyboardInterrupt: 

In [13]:
unique_articles

[['Why I’m not doing anything to cope with lower interest rates',
  'How should a retail investor deal with Wednesday’s interest rate cut by the Federal Reserve and with the future rate cuts that seem to be on the horizon?',
  'Yahoo Finance',
  '5 hours ago',
  [['^GSPC', '-0.19%'], ['BRK-B', '-0.96%']]],
 ['Analyst outlines best options market plays post-triple witching',
  'Friday’s market close marked the end of a “triple witching” session where $5.1 trillion in stock index futures, stock index options, and stock options expire simultaneously. September’s triple witching comes the same week as the Federal Reserve kicks off its rate-cutting cycle and a rebalancing of the S&P 500 (^GSPC).&nbsp; eToro US investment and options analyst Bret Kenwell joins Julie Hyman and Josh Lipton to discuss the options playbook after the big day for options traders. “Today was one of the four big expiration dates of the year when it comes to the options market, and you saw that action leading up to t

In [9]:
len(unique_articles)

320

In [10]:
pd.DataFrame(unique_articles, columns=['title', 'subtitle', 'publisher', 'date', 'taxonomies'])

,title,subtitle,publisher,date,taxonomies
0,Why I’m not doing anything to cope with lower ...,How should a retail investor deal with Wednesd...,Yahoo Finance,5 hours ago,"[[^GSPC, -0.19%], [BRK-B, -0.96%]]"
1,Analyst outlines best options market plays pos...,Friday’s market close marked the end of a “tri...,Yahoo Finance Video,23 hours ago,"[[QQQ, -0.19%], [^DJI, +0.09%], [XLK, -0.49%]]"
2,This Monster Growth Stock Is Up Nearly 300% in...,I believe the stock can keep rising faster tha...,Motley Fool,4 hours ago,"[[MELI, ], [^GSPC, -0.19%]]"
3,12 Stocks to Buy Now That the Fed Is Cutting R...,"Cyclicals, small caps and, yes, even some tech...",Barrons.com,6 hours ago,"[[^GSPC, -0.19%]]"
4,"Krystal Biotech, Inc. (KRYS): Promising Pipeli...",We recently published a list of 10 Worst Boomi...,Insider Monkey,6 hours ago,"[[KRYS, ], [COMP, ], [^GSPC, -0.19%]]"
...,...,...,...,...,...
315,"Revolve Group, Inc. (RVLV): Short Seller Senti...",We recently compiled a list of the 14 Worst 52...,Insider Monkey,15 hours ago,"[[RVLV, ], [^GSPC, -0.19%]]"
316,"Rocket Lab USA, Inc. (RKLB): Short Seller Sent...",We recently compiled a list of the 14 Worst 52...,Insider Monkey,15 hours ago,"[[^GSPC, -0.19%]]"
317,VIZIO Holding Corp. (VZIO): Short Seller Senti...,We recently compiled a list of the 14 Worst 52...,Insider Monkey,15 hours ago,"[[VZIO, ], [^GSPC, -0.19%]]"
318,Tecnoglass Inc. (TGLS): Short Seller Sentiment...,We recently compiled a list of the 14 Worst 52...,Insider Monkey,15 hours ago,"[[^GSPC, -0.19%]]"
